<a href="https://colab.research.google.com/github/longthanhta/AI_Chem/blob/main/bttc_pagerank_cloud_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import networkx as nx # network
from tqdm.auto import tqdm # loading bar
import scipy.stats as stats # KTC
import numpy as np
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200
tqdm.pandas()
G=nx.DiGraph()
import math


## Import file

In [ ]:
from google.colab import drive
import glob
drive.mount("/content/gdrive")
gg_drive_dataset_path="/content/gdrive/MyDrive/Octane - repository/Data/BTTC_april_2023/" # path to database in ggdrive
for item in (glob.glob(gg_drive_dataset_path+'*')): # check dataset
  print(item)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Octane - repository/Data/BTTC_april_2023/test.csv
/content/gdrive/MyDrive/Octane - repository/Data/BTTC_april_2023/bttc_16000000_19509379.csv
/content/gdrive/MyDrive/Octane - repository/Data/BTTC_april_2023/Tuan Sent
/content/gdrive/MyDrive/Octane - repository/Data/BTTC_april_2023/raw


In [ ]:
file_name='bttc_16000000_19509379.csv'
df=pd.read_csv(gg_drive_dataset_path+'/'+file_name,low_memory=True)

In [ ]:
df.head()

,block_number,from_address,to_address,gas,gas_used,gas_price,input,value,token_transfer,token_contract
0,16000169,0x4f2d66999f45483df5e5b309f1c959cbf0d1e480,0x746697d9ad21fb9c6ed63b11c81cb66eaba48d71,126697,69465,300000000000000,0x4f74acfe,10000000000000000000,0,0x
1,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x05bdc6c937d318a85277e1a97e11073b21a5fc02,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1061000000000000000000000,0,0x
2,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x0712b30476fb53f5c81ae1440308547438f2f7c5,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1060000000000000000000000,0,0x
3,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x07b168288b158ea79f4736bed17138b405de6223,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,553000000000000000000000,0,0x
4,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x0b5a49253c76d872b51a5b3eaca0ccbfc21f330c,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1060000000000000000000000,0,0x


In [ ]:
df = df[(df['from_address']!=df['to_address'])]# remove loop
df = df.drop_duplicates(subset=['block_number','from_address','to_address'])# remove duplicate 



Native value is Nan: using other tokens </br>
Native value is not Nane, using bttc </br>
Can currently ignore input col  </br>
Token constract:  0x1249c65afb11d179ffb3ce7d4eedd1d9b98ad00 is enthereum (for example)



In [ ]:
df.head()

,block_number,from_address,to_address,gas,gas_used,gas_price,input,value,token_transfer,token_contract
0,16000169,0x4f2d66999f45483df5e5b309f1c959cbf0d1e480,0x746697d9ad21fb9c6ed63b11c81cb66eaba48d71,126697,69465,300000000000000,0x4f74acfe,10000000000000000000,0,0x
1,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x05bdc6c937d318a85277e1a97e11073b21a5fc02,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1061000000000000000000000,0,0x
2,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x0712b30476fb53f5c81ae1440308547438f2f7c5,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1060000000000000000000000,0,0x
3,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x07b168288b158ea79f4736bed17138b405de6223,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,553000000000000000000000,0,0x
4,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x0b5a49253c76d872b51a5b3eaca0ccbfc21f330c,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1060000000000000000000000,0,0x


## Simple weight calculating

In [ ]:
origin = 16000000
lastest= 19509379
df['weight']=(df['block_number']-16000000)/(lastest-origin)

<ipython-input-39-90d1d6b79a7f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['weight']=(df['block_number']-16000000)/(lastest-origin)


In [ ]:
df.head()

,block_number,from_address,to_address,gas,gas_used,gas_price,input,value,token_transfer,token_contract,weight
0,16000169,0x4f2d66999f45483df5e5b309f1c959cbf0d1e480,0x746697d9ad21fb9c6ed63b11c81cb66eaba48d71,126697,69465,300000000000000,0x4f74acfe,10000000000000000000,0,0x,0.000048
1,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x05bdc6c937d318a85277e1a97e11073b21a5fc02,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1061000000000000000000000,0,0x,0.000310
2,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x0712b30476fb53f5c81ae1440308547438f2f7c5,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1060000000000000000000000,0,0x,0.000310
3,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x07b168288b158ea79f4736bed17138b405de6223,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,553000000000000000000000,0,0x,0.000310
4,16001088,0x0b6de2fb2873cbf6220784bdb70f9d4388553621,0x0b5a49253c76d872b51a5b3eaca0ccbfc21f330c,300000,21368,300000000000000,0x46696c65537061636541776172643a3230323231323035,1060000000000000000000000,0,0x,0.000310


## Dataframe to graph(networkx)


In [ ]:
G=nx.from_pandas_edgelist(df, 'from_address', 'to_address', create_using=nx.DiGraph, edge_attr='weight')


In [ ]:
print(len(df),'rows into',len(G.nodes()),'nodes and',len(G.edges()),'edge')

6433040 rows into 2298333 nodes and 4168990 edge


In [ ]:
node_set=list(G.nodes())
len(node_set)

2298333

## Pagerank calculation

In [ ]:
pr=nx.pagerank(G,0.5 ,max_iter=10000000000)
# dict to panda for ranking
pr_df=pd.DataFrame(pr.items(),columns=['add','pr'])
pr_df=pr_df.sort_values('pr',ascending=False)

In [ ]:
# scale the result
pr_df['pr']=pr_df['pr']*1000000
pr_df['pr_log']=np.log(pr_df['pr'])*1000


pr_df.head()



In [ ]:
with open('/content/gdrive/My Drive/file.txt', 'w') as f:
  pr_df.to_csv('pagerank.csv')

In [ ]:
ax = pr_df['pr_log'].plot.hist(bins=100)
ax.grid()
figure(figsize=(4, 4))

ax.set_yscale('log')

ax.set_ylabel('Count (log scale)')
ax.set_xlabel('Reputaiton score (x10e6)')
ax.set_title('Reputation Distribution')

plt.show()

In [ ]:
out_df=pd.DataFrame(out_table)
out_dv.to_csv(gg_drive_dataset_path+'out.csv')

## Plotting

## Kendall Tau Correlation

In [ ]:
'''
# Get Indegree
ind_list=[]
for add in add_list:
    indegree=G.in_degree(add)
    ind_list.append(indegree)

# dict to panda for ranking
ind_df=pd.DataFrame(dict(add=add_list, ind=ind_list),columns=['add','ind'])
ind_df=ind_df.sort_values('ind',ascending=False)
# reset index to get ranking
ind_df=ind_df.reset_index(drop=True)
# for calculate kentau coeficent
ind_df.reset_index(inplace=True)
ind_df = ind_df.rename(columns = {'index':'rank'})

ind_df.head()

pr_df=pr_df.sort_values('add')
pr_ranks=pr_df['rank']
ind_df=ind_df.sort_values('add')
ind_ranks=ind_df['rank']
tau, p_value = stats.kendalltau(pr_ranks, ind_ranks)
print('ktc',tau)
'''